In [47]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.precision', 6)

In [3]:
data = pd.read_excel("2024_cohorts_merged_edited_copy.xlsx", sheet_name="2019_2022_merged", header=5)

In [121]:
# data = pd.read_excel("2024_cohorts_merged_edited_copy.xlsx", sheet_name="2019_2022_merged", header=5, usecols=['JEW:JFG', 'JQS'])

In [122]:
# data = pl.read_excel("2024_cohorts_merged_edited.xlsx", sheet_name="2019_2022_merged")

In [104]:
df = data.copy()

In [105]:
# df.to_csv("Combined_2024_merged.csv", index=False)

In [106]:
df['pre_district'] = df['pre_district'].replace({"kanungu":"Kanungu"})

In [107]:
df['pre_district'].value_counts()

pre_district
Kanungu            2942
Mitooma            2084
Rubanda            1902
Kaliro             1603
Rukungiri          1566
Rubirizi           1089
Kyenjojo           1005
Kagadi_GAC          601
Kibaale             583
Kagadi_Standard     470
Kisoro              169
Name: count, dtype: int64

In [108]:
df['pre_cohort'].value_counts()

pre_cohort
2022    11344
2019     2670
Name: count, dtype: int64

In [109]:
df.shape

(14014, 7221)

In [114]:
df["Total Food Consumption"] = df[
                                    ["consumption_own_production.3","consumption_exp_weekly(ugx).2"]
                                ].sum(axis=1)

In [118]:
year_2 = df[df['pre_cohort'] == 2022]
year_5 = df[df['pre_cohort'] == 2019]

## Year 2 2022 cohort

In [80]:
year_2.shape

(11344, 7225)

In [81]:
year_2['STATUS'].value_counts()

STATUS
RTV     8164
PEER    3180
Name: count, dtype: int64

In [82]:
year_2.groupby('STATUS')['Farm Expenses'].mean()

STATUS
PEER    21890.566038
RTV     34542.950514
Name: Farm Expenses, dtype: float64

## Spending Adjusted

In [110]:
# variables to change:
df["expenditure_Medical_care_new"] = df["expenditure_Medical_care"] / 12
df["expenditure_Gifts_new"] = df["expenditure_Gifts"] / 12
df["expenditure_Recreation_new"] = df["expenditure_Recreation"] / 12

In [70]:
# annual_variables = [
#     'expenditure_durable_products',
#     'expenditure_clothing_women',
#     'expenditure_clothing_men',
#     'expenditure_clothing_children_girls',
#     'expenditure_clothing_children_boys',
#     'expenditure_Medical_care',
#     'expenditure_Maintenance_house',
#     'expenditure_Improvements_home',
#     'expenditure_Household_items',
#     'expenditure_Gifts',
#     'expenditure_Recreation'
# ]


In [111]:
annual_variables = [
    'expenditure_durable_products',
    'expenditure_clothing_women',
    'expenditure_clothing_men',
    'expenditure_clothing_children_girls',
    'expenditure_clothing_children_boys',
    'expenditure_Maintenance_house',
    'expenditure_Improvements_home',
    'expenditure_Household_items',
]


In [66]:
(df[monthly_variables].sum(axis=1) * 12).mean()

273818.64078778366

In [68]:
(df[annual_variables].sum(axis=1)).mean()

142955.29042386185

In [72]:
# monthly_variables = [
#     'expenditure_Fuel_lighting',
#     'expenditure_Utilities',
#     'expenditure_Phone_credit',
#     'expenditure_Transport',
#     'expenditure_washing_cleaning_products',
#     'expenditure_Rent_paid_homestead'
# ]

In [112]:
monthly_variables = [
    'expenditure_Fuel_lighting',
    'expenditure_Utilities',
    'expenditure_Phone_credit',
    'expenditure_Transport',
    'expenditure_washing_cleaning_products',
    'expenditure_Rent_paid_homestead',
    'expenditure_Medical_care_new',
    'expenditure_Gifts_new',
    'expenditure_Recreation_new'   
]

In [113]:
df["consumption_monthly_changed"] = (df[monthly_variables].sum(axis=1) * 12)

In [116]:
(year_2.groupby('STATUS')[
     'consumption_monthly_changed'
 ]).mean() * (inflation_adjustment / 100) / const_rate_adjustment

STATUS
PEER    60.284881
RTV     68.176048
Name: consumption_monthly_changed, dtype: float64

In [117]:
df["consumption_annual_changed"] = df[annual_variables].sum(axis=1)

In [119]:
(year_2.groupby('STATUS')[
     'consumption_annual_changed'
 ]).mean() * (inflation_adjustment / 100) / const_rate_adjustment

STATUS
PEER    21.077126
RTV     24.747968
Name: consumption_annual_changed, dtype: float64

In [120]:
(year_2[
    ["consumption_exp_weekly(ugx)", "consumption_monthly_changed" ,"consumption_annual_changed"]
]).sum(axis=1).mean()

1586758.802538787

In [121]:
(year_2[
    ["consumption_exp_weekly(ugx)", "consumption_exp_monthly" ,"consumption_exp _annual"]
]).sum(axis=1).mean()

1586758.802538787

## Spending And Expenditure

"consumption_exp_weekly(ugx)"	"consumption_exp_monthly"	"consumption_exp _annual"

In [15]:
inflation_adjustment = 78.08 
const_rate_adjustment = 3600

In [16]:
    [
        "Total Expenses.3", "Education Expenses.3", "Farm Expenses.3",
        "consumption_own_production.3","Total Food Consumption","Total crop_consumption.3",
        "consumption_exp_weekly(ugx).2", "consumption_exp_monthly.2", "consumption_exp _annual.2"
    ]

['Total Expenses.3',
 'Education Expenses.3',
 'Farm Expenses.3',
 'consumption_own_production.3',
 'Total Food Consumption',
 'Total crop_consumption.3',
 'consumption_exp_weekly(ugx).2',
 'consumption_exp_monthly.2',
 'consumption_exp _annual.2']

In [17]:
year_2.groupby('STATUS')[
    [
        "Total Expenses.3", "Education Expenses.3", "Farm Expenses.3",
        "consumption_own_production.3","Total Food Consumption","Total crop_consumption.3"
    ]
].mean().transpose()

STATUS,PEER,RTV
Total Expenses.3,554.956876,731.037593
Education Expenses.3,111.075484,122.606513
Farm Expenses.3,4.747821,7.491982
consumption_own_production.3,134.387468,241.440200
Total Food Consumption,357.771565,508.015081
Total crop_consumption.3,439.133572,600.939098


In [139]:
def calculate_mean_by_status(data, column):
    return data[data[column] != 0].groupby('STATUS')[column].agg(['mean', 'median'])

In [141]:
calculate_mean_by_status(year_2, "Farm Expenses.3")

,mean,median
STATUS,,
PEER,6.800932,4.120889
RTV,9.219859,6.506667


In [143]:
year_2['Farm Expenses.3'].apply(lambda x: 1 if x>0 else 0).value_counts(normalize=True)

Farm Expenses.3
1    0.780501
0    0.219499
Name: proportion, dtype: float64

In [140]:
year_2.groupby('STATUS')['Farm Expenses.3'].mean()

STATUS
PEER    4.747821
RTV     7.491982
Name: Farm Expenses.3, dtype: float64

In [18]:
year_2.groupby('STATUS')[["consumption_exp_weekly(ugx).2",
                          "consumption_exp_monthly.2",
                          "consumption_exp _annual.2"]].mean().transpose()

STATUS,PEER,RTV
consumption_exp_weekly(ugx).2,223.384097,266.574882
consumption_exp_monthly.2,53.242791,60.769885
consumption_exp _annual.2,28.119215,32.154132


In [19]:
df["Education Expenses.3"].mean(), df["Farm Expenses.3"].mean()

(120.42737757480614, 6.799329838161839)

In [20]:
year_2.groupby('STATUS')[
    ['Education Expenses.3', 'Farm Expenses.3']
].mean()

,Education Expenses.3,Farm Expenses.3
STATUS,,
PEER,111.075484,4.747821
RTV,122.606513,7.491982


## Expenditure Items

In [43]:
expenditure_items_columns = [
    'cereals_week',
    'tubers_week',
    'pulses_week',
    'milk_week',
    'vegetables_week',
    'fruits_week',
    'meat_poultry_offals',
    'eggs',
    'fish_week',
    'sugar_week',
    'fat_oils_week',
    'misc_week',
    'snacks_week',
    'alcohol_week'
]

result = (year_2.groupby('STATUS')[expenditure_items_columns]
          .mean() * (inflation_adjustment / 100) / const_rate_adjustment).transpose()
result

STATUS,PEER,RTV
cereals_week,0.733507,0.747903
tubers_week,0.335107,0.249560
pulses_week,0.442160,0.394101
milk_week,0.091407,0.232547
vegetables_week,0.272496,0.119007
fruits_week,0.066506,0.071352
meat_poultry_offals,0.587714,1.059896
eggs,0.075754,0.142625
fish_week,0.454682,0.682467
sugar_week,0.298147,0.490093


## consumed_not_spent

In [ ]:
consumed_not_spent = year_2[(year_2['cereals_week'] == 0) & (year_2['cereals_days'] > 0)]

grouped_result = consumed_not_spent.groupby(['STATUS', 'cereals_days']).size()

grouped_result

In [164]:
def analyze_consumption_without_spending(df, item):
    consumed_not_spent = df[(df[f'{item}_week'] == 0) & (df[f'{item}_days'] > 0)]
    return consumed_not_spent.groupby(['STATUS', f'{item}_days']).size()

In [167]:
items = ['cereals', 'vegetables', 'fruits']
results = {item: analyze_consumption_without_spending(year_2, item) for item in items}
results

{'cereals': STATUS  cereals_days
 PEER    1                36
         2               144
         3               231
         4               144
         5                95
         6                96
         7               105
 RTV     1                84
         2               479
         3               845
         4               391
         5               415
         6               339
         7               445
 dtype: int64,
 'vegetables': STATUS  vegetables_days
 PEER    1                    48
         2                   200
         3                   257
         4                   122
         5                   162
         6                    98
         7                    94
 RTV     1                   123
         2                   803
         3                  1468
         4                   715
         5                   841
         6                   540
         7                   649
 dtype: int64,
 'fruits': STATUS  fruits_days

## continuation

In [22]:
# Create a copy of the data including the STATUS column
year_2_filtered = year_2[['STATUS'] + expenditure_items_columns].copy()

# Replace zeros with NaN only for the expenditure columns, not STATUS
year_2_filtered[expenditure_items_columns] = year_2_filtered[expenditure_items_columns].replace(0, np.nan)

In [23]:
# Filtering out the zeros

In [24]:
(year_2_filtered.groupby('STATUS')[expenditure_items_columns]
          .mean() * (inflation_adjustment / 100) / const_rate_adjustment).transpose()

STATUS,PEER,RTV
cereals_week,1.364864,1.442788
tubers_week,1.578726,1.500303
pulses_week,1.495818,1.572550
milk_week,0.768980,0.950684
vegetables_week,0.678041,0.537673
fruits_week,0.556548,0.630433
meat_poultry_offals,2.163115,2.267555
eggs,0.651077,0.678154
fish_week,1.980671,2.065107
sugar_week,0.677220,0.723923


In [25]:
(year_2.groupby('STATUS')[['alcohol_week',
                        'tobacco_week', 
                        'consump_food_gift_week', 
                        'consump_inkind_week'
                          ]].mean() * (inflation_adjustment/100) /const_rate_adjustment).transpose()

STATUS,PEER,RTV
alcohol_week,0.173320,0.158985
tobacco_week,0.004501,0.003932
consump_food_gift_week,0.054406,0.069942
consump_inkind_week,0.047879,0.046367


In [26]:
!pip install -Uq scipy

In [27]:
import pandas as pd
import scipy.stats as stats


outcome_vars =  [
        "Total Expenses.3", "Education Expenses.3", "Farm Expenses.3",
        "consumption_own_production.3","Total Food Consumption","Total crop_consumption.3",
        "consumption_exp_weekly(ugx).2", "consumption_exp_monthly.2", "consumption_exp _annual.2"
    ]

# Loop through each outcome variable
for outcome in outcome_vars:
    treated = year_2[year_2['STATUS'] == 'RTV'][outcome]
    control = year_2[year_2['STATUS'] == 'PEER'][outcome]
    
    # Perform the t-test
    t_stat, p_value = stats.ttest_ind(treated, control)
    
    print(f"Outcome: {outcome}")
    print(f"T-statistic: {t_stat}, P-value: {p_value}")
    
    if p_value < 0.05:
        print(f"The treatment effect on {outcome} is statistically significant.\n")
    else:
        print(f"The treatment effect on {outcome} is not statistically significant.\n")


Outcome: Total Expenses.3
T-statistic: 30.4082018601314, P-value: 2.5924008350910845e-195
The treatment effect on Total Expenses.3 is statistically significant.

Outcome: Education Expenses.3
T-statistic: 4.168872487600474, P-value: 3.083842668001174e-05
The treatment effect on Education Expenses.3 is statistically significant.

Outcome: Farm Expenses.3
T-statistic: 14.036739760275871, P-value: 2.1830564472680165e-44
The treatment effect on Farm Expenses.3 is statistically significant.

Outcome: consumption_own_production.3
T-statistic: 37.00567583330302, P-value: 4.154241911384237e-283
The treatment effect on consumption_own_production.3 is statistically significant.

Outcome: Total Food Consumption
T-statistic: 34.56390630232134, P-value: 5.491446104803251e-249
The treatment effect on Total Food Consumption is statistically significant.

Outcome: Total crop_consumption.3
T-statistic: 34.481633154823434, P-value: 7.19205036301067e-248
The treatment effect on Total crop_consumption.3 i

In [41]:
1-(99/100), 1-(95/100), 1-(90/100)

(0.010000000000000009, 0.050000000000000044, 0.09999999999999998)

***   --- significant 99%
**    --- Significant 95%
*     --- Significant 90%

In [44]:
# Loop through each outcome variable
for outcome in expenditure_items_columns:
    treated = year_2[year_2['STATUS'] == 'RTV'][outcome]
    control = year_2[year_2['STATUS'] == 'PEER'][outcome]
    
    # Perform the t-test
    t_stat, p_value = stats.ttest_ind(treated, control)
    
    print(f"Outcome: {outcome}")
    print(f"T-statistic: {t_stat}, P-value: {p_value}")
    
    if p_value < 0.05:
        print(f"The treatment effect on {outcome} is statistically significant.\n")
    else:
        print(f"The treatment effect on {outcome} is not statistically significant.\n")


Outcome: cereals_week
T-statistic: 0.7233971272480489, P-value: 0.46945085371674844
The treatment effect on cereals_week is not statistically significant.

Outcome: tubers_week
T-statistic: -5.543589961199426, P-value: 3.0295864302494165e-08
The treatment effect on tubers_week is statistically significant.

Outcome: pulses_week
T-statistic: -2.7983843572444425, P-value: 0.005144561505441294
The treatment effect on pulses_week is statistically significant.

Outcome: milk_week
T-statistic: 14.333549577492267, P-value: 3.4166115424349153e-46
The treatment effect on milk_week is statistically significant.

Outcome: vegetables_week
T-statistic: -24.40695032677409, P-value: 2.8370379149429643e-128
The treatment effect on vegetables_week is statistically significant.

Outcome: fruits_week
T-statistic: 0.971136215087396, P-value: 0.33150116106433125
The treatment effect on fruits_week is not statistically significant.

Outcome: meat_poultry_offals
T-statistic: 18.190498419357592, P-value: 6.64

## Farm Expenses

1	Ploughs
2	Machetes/Panga
3	Hoes
4	Rent of tractors
5	Ox Ploughs
6	Watering Can
7	Seeds
8	Irrigation
97	Other (please specify)

1	Organic Pesticides
2	Organic Fungicides
3	Organic Herbicides
4	Synthetic Pesticides
5	Synthetic Fungicides
6	Synthetic Herbicides
99	None

In [133]:
farm_variables = [
    'farming_inputs1_expenses', 'farming_inputs2_expenses', 'farming_inputs3_expenses',
    'farming_inputs4_expenses', 'farming_inputs5_expenses', 'farming_inputs6_expenses',
    'farming_inputs7_expenses', 'farming_inputs8_expenses', 'farming_inputsother_expenses',
    'organic_inputs_1', 'organic_inputs_2', 'organic_inputs_3',
    'organic_inputs_4', 'organic_inputs_5', 'organic_inputs_6', 'organic_inputs_99',
    'organic1_expenses', 'organic2_expenses', 'organic3_expenses', 'organic4_expenses',
    'organic5_expenses', 'organic6_expenses', 'bags_storage_expenses',
    'formal_agriculturalinsurance_expenses', 'formal_transport_farm_expenses'
]


In [134]:
(year_2.groupby('STATUS')[farm_variables]
          .mean() * (inflation_adjustment / 100) / const_rate_adjustment).transpose()

STATUS,PEER,RTV
farming_inputs1_expenses,7.547733,7.757276
farming_inputs2_expenses,2.495688,2.801240
farming_inputs3_expenses,3.407583,3.643724
farming_inputs4_expenses,7.536889,11.526095
farming_inputs5_expenses,7.343073,6.365448
farming_inputs6_expenses,2.618159,2.293519
farming_inputs7_expenses,4.726370,5.154837
farming_inputs8_expenses,1.616808,2.472786
farming_inputsother_expenses,1.756800,1.554370
organic_inputs_1,0.241974,0.476991


In [137]:
# Loop through each outcome variable
for outcome in farm_variables:
    treated = year_2[year_2['STATUS'] == 'RTV'][outcome].dropna()
    control = year_2[year_2['STATUS'] == 'PEER'][outcome].dropna()
    
    # Perform the t-test
    t_stat, p_value = stats.ttest_ind(treated, control)
    
    print(f"Outcome: {outcome}")
    print(f"T-statistic: {t_stat}, P-value: {p_value}")
    
    if p_value < 0.05:
        print(f"The treatment effect on {outcome} is statistically significant.\n")
    else:
        print(f"The treatment effect on {outcome} is not statistically significant.\n")


Outcome: farming_inputs1_expenses
T-statistic: 0.3541310204781425, P-value: 0.7237314689587887
The treatment effect on farming_inputs1_expenses is not statistically significant.

Outcome: farming_inputs2_expenses
T-statistic: 3.518172672251375, P-value: 0.00044637246070644364
The treatment effect on farming_inputs2_expenses is statistically significant.

Outcome: farming_inputs3_expenses
T-statistic: 3.1646650973390553, P-value: 0.0015693689932048743
The treatment effect on farming_inputs3_expenses is statistically significant.

Outcome: farming_inputs4_expenses
T-statistic: 1.0658649580074, P-value: 0.3218615283420938
The treatment effect on farming_inputs4_expenses is not statistically significant.

Outcome: farming_inputs5_expenses
T-statistic: -3.018912330050953, P-value: 0.002742759650483662
The treatment effect on farming_inputs5_expenses is statistically significant.

Outcome: farming_inputs6_expenses
T-statistic: -1.026594696699706, P-value: 0.306309190715452
The treatment effe

## Year 5

In [28]:
year_5.groupby('STATUS')[
    ["Education Expenses.3","Farm Expenses.3"]
].mean()

,Education Expenses.3,Farm Expenses.3
STATUS,,
RTV,124.902493,7.124793


In [124]:
year_five_control = year_2[(year_2["pre_district"] == "Kanungu") & (year_2["STATUS"] == "PEER")]
year_five_control.shape

(376, 7227)

In [125]:
year_5.shape[0] + year_five_control.shape[0]

3046

In [126]:
year_5 = pd.concat([year_5, year_five_control])

In [127]:
year_5['STATUS'].value_counts()

STATUS
RTV     2670
PEER     376
Name: count, dtype: int64

In [128]:
year_5.groupby('STATUS')[
    [
        "Total Expenses.3", "Education Expenses.3", "Farm Expenses.3",
        "consumption_own_production.3","Total Food Consumption","Total crop_consumption.3"
    ]
].mean().transpose()

STATUS,PEER,RTV
Total Expenses.3,569.902126,727.146428
Education Expenses.3,92.890165,124.902493
Farm Expenses.3,5.173723,7.124793
consumption_own_production.3,176.943755,270.266323
Total Food Consumption,394.978002,501.592814
Total crop_consumption.3,471.838237,595.090326


In [129]:
year_5.groupby('STATUS')[["consumption_exp_weekly(ugx).2",
                          "consumption_exp_monthly.2",
                          "consumption_exp _annual.2"]].mean().transpose()

STATUS,PEER,RTV
consumption_exp_weekly(ugx).2,218.034247,231.528938
consumption_exp_monthly.2,48.548732,62.482813
consumption_exp _annual.2,28.311503,30.930501


In [130]:
(year_5.groupby('STATUS')[
     'consumption_monthly_changed'
 ]).mean() * (inflation_adjustment / 100) / const_rate_adjustment

STATUS
PEER    55.003200
RTV     69.819751
Name: consumption_monthly_changed, dtype: float64

In [131]:
(year_5.groupby('STATUS')[
     'consumption_annual_changed'
 ]).mean() * (inflation_adjustment / 100) / const_rate_adjustment

STATUS
PEER    21.857035
RTV     23.593564
Name: consumption_annual_changed, dtype: float64

In [45]:
# Loop through each outcome variable
for outcome in outcome_vars:
    treated = year_5[year_5['STATUS'] == 'RTV'][outcome]
    control = year_5[year_5['STATUS'] == 'PEER'][outcome]
    
    # Perform the t-test
    t_stat, p_value = stats.ttest_ind(treated, control)
    
    print(f"Outcome: {outcome}")
    print(f"T-statistic: {t_stat}, P-value: {p_value}")
    
    if p_value < 0.05:
        print(f"The treatment effect on {outcome} is statistically significant.\n")
    else:
        print(f"The treatment effect on {outcome} is not statistically significant.\n")


Outcome: Total Expenses.3
T-statistic: nan, P-value: nan
The treatment effect on Total Expenses.3 is not statistically significant.

Outcome: Education Expenses.3
T-statistic: 4.664313066619557, P-value: 3.2299320271733097e-06
The treatment effect on Education Expenses.3 is statistically significant.

Outcome: Farm Expenses.3
T-statistic: 4.050625688604819, P-value: 5.235637865535231e-05
The treatment effect on Farm Expenses.3 is statistically significant.

Outcome: consumption_own_production.3
T-statistic: nan, P-value: nan
The treatment effect on consumption_own_production.3 is not statistically significant.

Outcome: Total Food Consumption
T-statistic: 7.471092663054689, P-value: 1.0333333553732525e-13
The treatment effect on Total Food Consumption is statistically significant.

Outcome: Total crop_consumption.3
T-statistic: nan, P-value: nan
The treatment effect on Total crop_consumption.3 is not statistically significant.

Outcome: consumption_exp_weekly(ugx).2
T-statistic: 1.7044

In [46]:
# Loop through each outcome variable
for outcome in expenditure_items_columns:
    treated = year_5[year_5['STATUS'] == 'RTV'][outcome]
    control = year_5[year_5['STATUS'] == 'PEER'][outcome]
    
    # Perform the t-test
    t_stat, p_value = stats.ttest_ind(treated, control)
    
    print(f"Outcome: {outcome}")
    print(f"T-statistic: {t_stat}, P-value: {p_value}")
    
    if p_value < 0.05:
        print(f"The treatment effect on {outcome} is statistically significant.\n")
    else:
        print(f"The treatment effect on {outcome} is not statistically significant.\n")


Outcome: cereals_week
T-statistic: -0.062374495014457944, P-value: 0.9502687005466026
The treatment effect on cereals_week is not statistically significant.

Outcome: tubers_week
T-statistic: -1.9515774200055795, P-value: 0.0510799883217535
The treatment effect on tubers_week is not statistically significant.

Outcome: pulses_week
T-statistic: -0.7820492061364289, P-value: 0.4342464822405543
The treatment effect on pulses_week is not statistically significant.

Outcome: milk_week
T-statistic: 5.704386082987536, P-value: 1.2796359375657346e-08
The treatment effect on milk_week is statistically significant.

Outcome: vegetables_week
T-statistic: -31.22236954033585, P-value: 6.7512685757483325e-186
The treatment effect on vegetables_week is statistically significant.

Outcome: fruits_week
T-statistic: 0.5664156490734249, P-value: 0.5711530158273572
The treatment effect on fruits_week is not statistically significant.

Outcome: meat_poultry_offals
T-statistic: 4.885365214648341, P-value: 1

## Expenditure Items

In [201]:
expenditure_items_columns = [
    'cereals_week',
    'tubers_week',
    'pulses_week',
    'milk_week',
    'vegetables_week',
    'fruits_week',
    'meat_poultry_offals',
    'eggs',
    'fish_week',
    'sugar_week',
    'fat_oils_week',
    'misc_week',
    'snacks_week',
]

result_five = (year_5.groupby('STATUS')[expenditure_items_columns]
          .mean() * (inflation_adjustment / 100) / const_rate_adjustment).transpose()
result_five

STATUS,PEER,RTV
cereals_week,0.906030,0.902558
tubers_week,0.352098,0.277277
pulses_week,0.446180,0.412324
milk_week,0.057568,0.198612
vegetables_week,0.578620,0.095626
fruits_week,0.081333,0.090785
meat_poultry_offals,0.500402,0.829198
eggs,0.023362,0.092117
fish_week,0.116809,0.216361
sugar_week,0.244923,0.442689


In [37]:
# Loop through each outcome variable
for outcome in expenditure_items_columns:
    treated = year_5[year_5['STATUS'] == 'RTV'][outcome]
    control = year_5[year_5['STATUS'] == 'PEER'][outcome]
    
    # Perform the t-test
    t_stat, p_value = stats.ttest_ind(treated, control)
    
    print(f"Outcome: {outcome}")
    print(f"T-statistic: {t_stat}, P-value: {p_value}")
    
    if p_value < 0.05:
        print(f"The treatment effect on {outcome} is statistically significant.\n")
    else:
        print(f"The treatment effect on {outcome} is not statistically significant.\n")


Outcome: cereals_week
T-statistic: -0.062374495014457944, P-value: 0.9502687005466026
The treatment effect on cereals_week is not statistically significant.

Outcome: tubers_week
T-statistic: -1.9515774200055795, P-value: 0.0510799883217535
The treatment effect on tubers_week is not statistically significant.

Outcome: pulses_week
T-statistic: -0.7820492061364289, P-value: 0.4342464822405543
The treatment effect on pulses_week is not statistically significant.

Outcome: milk_week
T-statistic: 5.704386082987536, P-value: 1.2796359375657346e-08
The treatment effect on milk_week is statistically significant.

Outcome: vegetables_week
T-statistic: -31.22236954033585, P-value: 6.7512685757483325e-186
The treatment effect on vegetables_week is statistically significant.

Outcome: fruits_week
T-statistic: 0.5664156490734249, P-value: 0.5711530158273572
The treatment effect on fruits_week is not statistically significant.

Outcome: meat_poultry_offals
T-statistic: 4.885365214648341, P-value: 1

In [202]:
(year_5.groupby('STATUS')[['alcohol_week',
                        'tobacco_week', 
                        'consump_food_gift_week', 
                        'consump_inkind_week'
                          ]].mean() * (inflation_adjustment/100) /const_rate_adjustment).transpose()

STATUS,PEER,RTV
alcohol_week,0.244865,0.218189
tobacco_week,0.004326,0.003331
consump_food_gift_week,0.074988,0.084546
consump_inkind_week,0.056241,0.066813


In [203]:
# Create a copy of the data including the STATUS column
year_5_filtered = year_5[['STATUS'] + expenditure_items_columns].copy()

# Replace zeros with NaN only for the expenditure columns, not STATUS
year_5_filtered[expenditure_items_columns] = year_5_filtered[expenditure_items_columns].replace(0, np.nan)

In [148]:
# Filtering out the zeros

In [204]:
(year_5_filtered.groupby('STATUS')[expenditure_items_columns]
          .mean() * (inflation_adjustment / 100) / const_rate_adjustment).transpose()

STATUS,PEER,RTV
cereals_week,1.413558,1.565842
tubers_week,1.614500,1.322015
pulses_week,1.446238,1.446657
milk_week,0.746397,0.908037
vegetables_week,0.929749,0.526436
fruits_week,0.650667,0.824473
meat_poultry_offals,2.138081,2.360297
eggs,0.488000,0.681307
fish_week,1.996364,2.093056
sugar_week,0.618061,0.717220
